# What is Regression?

- Regression is a supervised learning technique
    - Feed features and answer in 
        - get model/algorithm out
- Target is a continuous variable 
    - for example, how much something costs
    - New evaluation metrics, like RMSE (root mean squared error)
- Same data structure
    - one column - one feature, one row - one observation
- Simple regression 
    - One feature, linear
- Multiple regression 
    - multiple features, linear
- Polynomial regression 
    - non-linear        

# Acquire

- SQL
    - Writing queries
    - Using pd.read_sql
    - Some function to generate server url
- Get to know the data
    - .info, .describe, .head, .shape
    - Histograms, countplots

# Prepare

- Null values
    - Investigate nulls 
    - Determine meaning
    - Take action (drop, impute)
    - .dropna, .isna, .value_counts
- Clean data
    - Encode categoricals as dummies (pd.get_dummies( ))
    - Change data types (.astype(
    ))
    - Tidy data
        - One observation in each row
        - One feature in each column
- Handle Outliers
    - Drop
    - Cap
- Split our data
    - train_test_split
        - test subests of original data

# Explore

- Ask questions of our data
    - Is x causing y?
- Build visualizations
- Use statistics to determine significance

# Model

- Scale the data
    - MinMaxScaler
        - makes all numbers fall between 0 and 1
    - StandardScaler 
        - makes all numbers the number of standard deviations from the mean 
    - Do not explore on scaled data
    - Do not scale target variable
- Feature engineering
    - Encode patterns
    - Combine two or more fields
        - Especially if they're highly correlated
- Feature Selection
    - RFE (recursive feature elimination)
    - Select KBest
    - Curse of Dimensionality
        - The idea that if the dimensions of your dataframe extend out significantly, the less effective the model can be
- Actual Modeling
    - Fit
    - Predict
    - Evaluate (RMSE)